# Dynamic Programming

> $\textcolor{#FFC0D9}{\Longrightarrow}$ The following section uses the book {cite}`clrs`, and the tutorials {cite}`geeksforgeeks_01knapsack, neetcode_advanced_algorithms, stackademic_knapsack` as a reference.

A dynamic-programming algorithm solves each subproblem just once and then saves its answer in a table, thereby avoiding the work of recomputing the answer every time it solves each subproblem. Dynamic programming applies when the subproblems overlap, that is, when subproblems share subsubproblems. In this context, 
<span style="background-color: #F9F9E0;">a divide-and-conquer algorithm does more work than necessary</span>, repeatedly solving the common subproblems. When developing a dynamic-programming algorithm, we follow a sequence of four steps:
1. Characterize the structure of an optimal solution
2. Recursively define the value of an optimal solution
3. Compute the value of an optimal solution, typically in a bottom-up fashion
4. Construct an optimal solution from computed information

Solving the problem initially using divide and conquer is not a bad idea, considering that we optimize the code logic afterwards.

## 0/1 Knapsack

**Example:** Imagine we are going to the supermarket, and we want to buy a selection of products to take with us to the park. We have only one backpack with a certain capacity, and we want to maximize the total value of the products we can fit into our backpack. The condition here is that we can only choose one product of each type.

We enter a supermarket with a backpack that has a capacity of $ m = 50 $ units. There are $ n = 3 $ products available:
- Product 1: cost $ c_1 = 60 $, weight $ w_1 = 10 $
- Product 2: cost $ c_2 = 100 $, weight $ w_2 = 20 $
- Product 3: cost $ c_3 = 120 $, weight $ w_3 = 30 $

The goal is to determine the maximum value of the backpack when products are not allowed to be fractioned. In this example, we should take Product 2 and Product 3 to maximize the total value. Thus, the total value would be 220.

## Algorithms

There are several methods to solve the knapsack problem. With that in mind, let's demonstrate some algorithms and illustrate each using the following:
- $weights = [1, 2, 3]$
- $values = [6, 10, 12]$
- $capacity = 5$

### Naive Algorithm
This algorithm has a time complexity of $O(2^n)$.


In [6]:
def knapsack_helper(level, capacity, weights, values):
    if level == len(values):
        return 0

    # Exclude the item, as including it would exceed the capacity of our backpack.
    if weights[level] > capacity:
        return knapsack_helper(level + 1, capacity, weights, values)
    
    profit_with = values[level] + knapsack_helper(level + 1, capacity - weights[level], weights, values)
    profit_without = knapsack_helper(level + 1, capacity, weights, values)

    return max(profit_with, profit_without)

def knapsack_naive(capacity, weights, values):
    return knapsack_helper(0, capacity, weights, values)

# Example usage:
weights = [1, 2, 3]
values = [6, 10, 12]
capacity = 5
print("Answer: ", knapsack_naive(capacity, weights, values))


Answer:  22


### Memoization Algorithm

Memoization is an optimization technique used primarily to speed up computer programs by storing the results of expensive function calls and reusing them when the same inputs occur again. This avoids the need to recompute results for the same inputs, significantly improving the time complexity in many cases. 

#### Key Concepts of Memoization

1. **Cache Storage:**
   Memoization involves creating a cache (typically a dictionary or array) where the results of function calls are stored. The cache keys are the function arguments, and the cache values are the computed results.

2. **Function Modification:**
   The original function is modified to first check if the result for a given input is already in the cache. If it is, the cached result is returned immediately, bypassing the computation. If not, the function computes the result, stores it in the cache, and then returns it.

3. **Time Complexity Improvement:**
   By avoiding redundant calculations, memoization can transform algorithms with exponential time complexity into those with polynomial time complexity. This is particularly effective in dynamic programming, where overlapping subproblems are common.

#### Steps to Implement Memoization

1. **Identify the Overlapping Subproblems:**
   Determine the parts of the algorithm where the same calculations are repeated multiple times.

2. **Create a Cache:**
   Initialize a data structure to store the results of these subproblems. A dictionary is commonly used for its efficient look-up time.

3. **Modify the Function:**
   Before performing any calculation, the function should check if the result is already in the cache. If it is, return the cached result. If not, compute the result, store it in the cache, and then return it.


This optimizes the naive approach using a <span style="background-color: #F9F9E0;">memoization technique</span>, significantly improving the time complexity to $O(n \cdot m)$.


In [7]:
def knapsack_helper_memo(level, capacity, weights, values, memo):
    if level == len(values):
        return 0
    
    if (level, capacity) in memo:
        return memo[(level, capacity)]
    
    # Exclude the item, as including it would exceed the capacity of our backpack.
    if weights[level] > capacity:
        memo[(level, capacity)] = knapsack_helper_memo(level + 1, capacity, weights, values, memo)
        return memo[(level, capacity)]
    
    profit_with = values[level] + knapsack_helper_memo(level + 1, capacity - weights[level], weights, values, memo)
    profit_without = knapsack_helper_memo(level + 1, capacity, weights, values, memo)
    
    memo[(level, capacity)] = max(profit_with, profit_without)
    return memo[(level, capacity)]

def knapsack_memo(capacity, weights, values):
    memo = {}
    return knapsack_helper_memo(0, capacity, weights, values, memo)

# Example usage:
weights = [1, 2, 3]
values = [6, 10, 12]
capacity = 5
print("Answer: ", knapsack_memo(capacity, weights, values))


Answer:  22


### Dynamic Programming Algorithm

This approach uses a 2D array to compute the solution iteratively, also achieving a time complexity of $O(n \cdot m)$.

The table in Figure below depicts the dynamic programming table for a knapsack problem with weights [1, 2, 3], values [6, 10, 12], and a capacity of 5. In this table:
- The rows represent the index of items, including a row for no items (index 0, 1, or 2).
- The columns correspond to knapsack capacities ranging from 0 to the maximum capacity.
- Each cell $dp[i][w]$ shows the maximum profit that can be obtained using the first $i$ items with a knapsack capacity of $w$.

:::{figure-md} knapsack-dp
<img src="./figures/knapsack-dp.png" alt="fishy" class="mb-1" width="400px">

Dynamic programming table for the 0/1 Knapsack problem with weights [1, 2, 3], values [6, 10, 12], and capacity of 5. Each cell $dp[i][m]$ represents the maximum profit for the first $i$ items with a capacity $m = 5$
:::

In [8]:
def knapsack_dp(values, weights, capacity):
    N = len(values)
    M = capacity
    dp = [[0] * (M + 1) for _ in range(N)]
    
    # Fill the first row based on the first item's weight and value
    for c in range(M + 1):
        if weights[0] <= c:
            dp[0][c] = values[0]
    
    for i in range(1, N):
        for c in range(1, M + 1):
            skip = dp[i-1][c]
            include = 0
            if c >= weights[i]:
                include = values[i] + dp[i-1][c - weights[i]]
            dp[i][c] = max(include, skip)
    
    print("The dp arrray: ", dp)
    return dp[N-1][M]

# Example usage:
weights = [1, 2, 3]
values = [6, 10, 12]
capacity = 5
print("Answer: ", knapsack_dp(values, weights, capacity))


The dp arrray:  [[0, 6, 6, 6, 6, 6], [0, 6, 10, 16, 16, 16], [0, 6, 10, 16, 18, 22]]
Answer:  22


#### Animation for DP approach

For a step by step animation on how the dynamic solution for the 0/1 knapsack problem please refer to the slides below

<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vR7ODtgf40VbvLtTd61B5_RtEtYDdmOffHlm7gz5DZpag2O4QwODrz-VId1LZTaaP6wWy0KPSn8TyHX/embed?start=false&loop=false&delayms=5000" frameborder="0" width="820" height="500" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>